In [1]:
import configparser
import json

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import asyncio
from datetime import date, datetime

In [4]:
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError
from telethon.tl.functions.messages import (GetHistoryRequest)
from telethon.tl.types import (
PeerChannel
)

In [5]:
class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()

        if isinstance(o, bytes):
            return list(o)

        return json.JSONEncoder.default(self, o)

In [6]:
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = str(config['Telegram']['api_hash'])


phone = config['Telegram']['phone']
username = config['Telegram']['username']

link = config['Channels']['link']


file_name = str(config['Files']['file'])  + ".json"


In [22]:
phone

'+380678885481'

In [7]:
# Create the client and connect

client = TelegramClient(username, api_id, api_hash)


In [8]:
async def main(phone,link, file_name):
    await client.start()
    print("Client Created")
    # Авторизація
    if await client.is_user_authorized() == False:
        await client.send_code_request(phone)
        try:
            await client.sign_in(phone, input('Enter the code: '))
        except SessionPasswordNeededError:
            print("Password is needed")

    me = await client.get_me()



    my_channel = await client.get_entity(link)

    offset_id = 0
    limit = 100
    all_messages = []
    total_messages = 0
    total_count_limit = 1000
    # Зчитування повідомлень
    
    while True:
        print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
        history = await client(GetHistoryRequest(
            peer=my_channel,
            offset_id=offset_id,
            offset_date=None,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0
        ))
        if not history.messages:
            break
        messages = history.messages
        for message in messages:
            all_messages.append(message.to_dict())
        offset_id = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break
    # Запис в файл 
    with open(file_name, 'w') as outfile:
        json.dump(all_messages, outfile, cls=DateTimeEncoder)
    await client.disconnect()




In [9]:
client.loop.run_until_complete(main(phone,link,file_name))

Client Created
Current Offset ID is: 0 ; Total Messages: 0
Current Offset ID is: 7840 ; Total Messages: 100
Current Offset ID is: 7729 ; Total Messages: 200
Current Offset ID is: 7620 ; Total Messages: 300
Current Offset ID is: 7510 ; Total Messages: 400
Current Offset ID is: 7406 ; Total Messages: 500
Current Offset ID is: 7295 ; Total Messages: 600
Current Offset ID is: 7183 ; Total Messages: 700
Current Offset ID is: 7079 ; Total Messages: 800
Current Offset ID is: 6970 ; Total Messages: 900
